In [56]:
# Initial imports
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pandas as pd
import numpy as np

In [57]:
file = '../Resources/Contributions_2013.csv'
IC_2017 = pd.read_csv(file,index_col='CandidateName')
uncleaned = pd.read_csv(file,index_col=False)
IC_2017.head()

/Users/Abir/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Election,CandidateID,Date,Name,ContributerType,BoroughName,City,State,ZipCode,ContributionState,Amount,MatchAmt,PrevAmt,PayMethod,TotalAmount
CandidateName,,,,,,,,,,,,,,,,
"Coenen, Jr., Kevin P",0,2013,1286,1/25/2013,"Coenen, Kevin P",Candidate,Other,Nesconset,NY,11767.0,NY,10.0,0.0,0.0,Credit Card,10.0
"Coenen, Jr., Kevin P",1,2013,1286,4/17/2013,"Walter, Dorsey",Individual,Manhattan,New York,NY,10150.0,NaN,175.0,175.0,0.0,Credit Card,350.0
"Coenen, Jr., Kevin P",2,2013,1286,1/25/2013,"Coenen, Kevin P",Candidate,Other,Nesconset,NY,11767.0,NaN,-10.0,0.0,0.0,Unknown,-10.0
"Marks, Philip A",3,2013,1551,2/13/2012,"Marks, Philip",Candidate,Brooklyn,Brooklyn,NY,11236.0,NY,400.0,175.0,0.0,Check,575.0
"Marks, Philip A",4,2013,1551,2/28/2013,"Marks, Philip",Candidate,Brooklyn,Brooklyn,NY,11236.0,NY,100.0,0.0,400.0,Cash,500.0


In [58]:
IC_2017['ZipCode'] = IC_2017['ZipCode'].apply(str)

In [81]:
IC_2017_df = IC_2017.drop(columns=['ContributionState','PayMethod','Name','CandidateID','Unnamed: 0','Election','BoroughName','Amount','MatchAmt','PrevAmt'])
IC_2017_df.groupby(['CandidateName']).sum(['TotalAmount']).sort_values('TotalAmount',ascending=False)

,TotalAmount
CandidateName,
"de Blasio, Bill",1.497036e+07
"Quinn, Christine C",1.209154e+07
"Thompson, Jr., William C",6.111452e+06
"Lhota, Joseph J",4.981273e+06
"Hidary, Jack D",6.760667e+05
"Salgado, Erick J",4.948541e+05
"Liu, John C",4.045000e+05
"Weiner, Anthony D",2.301887e+05
"Albanese, Sal F",3.092800e+04


In [64]:
outcome = []
for i in uncleaned['CandidateName']:
    if i == 'de Blasio, Bill':
        outcome.append('W')
    else:
        outcome.append('L')

IC_2017_df['Outcome'] = outcome
IC_2017_df.head()

,Date,ContributerType,City,State,ZipCode,TotalAmount,Outcome
CandidateName,,,,,,,
"Coenen, Jr., Kevin P",1/25/2013,Candidate,Nesconset,NY,11767.0,10.0,L
"Coenen, Jr., Kevin P",4/17/2013,Individual,New York,NY,10150.0,350.0,L
"Coenen, Jr., Kevin P",1/25/2013,Candidate,Nesconset,NY,11767.0,-10.0,L
"Marks, Philip A",2/13/2012,Candidate,Brooklyn,NY,11236.0,575.0,L
"Marks, Philip A",2/28/2013,Candidate,Brooklyn,NY,11236.0,500.0,L


In [65]:
IC_2017_df = IC_2017_df.dropna()
IC_2017_df.isnull().sum()

Date               0
ContributerType    0
City               0
State              0
ZipCode            0
TotalAmount        0
Outcome            0
dtype: int64

In [66]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, chi2
le = LabelEncoder()
# IC_2017_df['CandidateName']= le.fit_transform(IC_2017_df['CandidateName'])
IC_2017_df['Date'] = le.fit_transform(IC_2017_df['Date'])
IC_2017_df['ContributerType'] = le.fit_transform(IC_2017_df['ContributerType'])
IC_2017_df['City'] = le.fit_transform(IC_2017_df['City'])
# IC_2017_df['Amount'] = le.fit_transform(IC_2017_df['Amount'])
IC_2017_df['State'] = le.fit_transform(IC_2017_df['State'])
IC_2017_df['ZipCode'] = le.fit_transform(IC_2017_df['ZipCode'])
# IC_2017_df['MatchAmt'] = le.fit_transform(IC_2017_df['MatchAmt'])
# IC_2017_df['PrevAmt'] = le.fit_transform(IC_2017_df['PrevAmt'])
IC_2017_df['TotalAmount'] = le.fit_transform(IC_2017_df['TotalAmount'])
IC_2017_df['Outcome'] = le.fit_transform(IC_2017_df['Outcome'])
IC_2017_df.head()

,Date,ContributerType,City,State,ZipCode,TotalAmount,Outcome
CandidateName,,,,,,,
"Coenen, Jr., Kevin P",68,0,1569,51,511,120,0
"Coenen, Jr., Kevin P",729,4,1596,51,71,397,0
"Coenen, Jr., Kevin P",68,0,1569,51,511,105,0
"Marks, Philip A",482,0,291,51,326,475,0
"Marks, Philip A",548,0,291,51,326,456,0


In [67]:
X = IC_2017_df.drop('Outcome',axis = 1)

In [68]:
y = IC_2017_df['Outcome']

In [69]:
# Check the balance of our target values
y.value_counts()

0    30435
1    17162
Name: Outcome, dtype: int64

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [71]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE

from imblearn.ensemble import BalancedRandomForestClassifier
random_forest = BalancedRandomForestClassifier(n_estimators = 100,random_state = 1)
random_forest = random_forest.fit(X_train, y_train)

In [72]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score
y_pred = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7608672235309233

In [73]:
# Display the confusion matrix
# YOUR CODE HERE
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[5912, 1722],
       [1078, 3188]])

In [74]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.77      0.75      0.81      0.76      0.58      7634
          1       0.65      0.75      0.77      0.69      0.76      0.58      4266

avg / total       0.78      0.76      0.76      0.77      0.76      0.58     11900

